Lorenz performance experiments

In [ ]:
import numpy as np
import torch
import sys
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from IPython.display import clear_output

sys.path.append('../')
from source.utils import simulate_lorenz_96, compare_graphs
import source.utils as utils
import source.NMC as models
import importlib

## SVAM

In [3]:
# LiNGAM / SVAM performance with sparse data
import warnings
warnings.filterwarnings("ignore")
for dt in [0.01,0.05,0.1,0.15,0.25]:
    perf = []
    for i in range(20):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC= simulate_lorenz_96(p, T=T, delta_t=dt,sigma=0.05, F=10)

        # format for NeuralODE
        times = np.linspace(0, T, num_points)
        times_np = np.hstack([times[:, None]])
        times = torch.from_numpy(times_np[:, :, None])
        data = torch.from_numpy(data[:, None, :].astype(np.float32))

        indeces = np.random.choice(range(T),int(T),replace=False)
        indeces = np.sort(indeces)

        from source.lingam import lingam_method
        importlib.reload(utils)
        graph = lingam_method(data[indeces].squeeze().detach())
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('Means and standard deviations for TPR, FDR and AUC with', dt, 'time interval')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     

Means and standard deviations for TPR, FDR and AUC with 0.01 time interval
[0.38       0.29340147 0.6864375 ] [0.02915476 0.11233716 0.01874188]
Means and standard deviations for TPR, FDR and AUC with 0.05 time interval
[0.4525     0.17427324 0.70007292] [0.03864906 0.07293103 0.01824972]
Means and standard deviations for TPR, FDR and AUC with 0.1 time interval
[0.58       0.12462248 0.69808333] [0.02806243 0.04960403 0.01512357]
Means and standard deviations for TPR, FDR and AUC with 0.15 time interval
[0.66625    0.24059382 0.64332292] [0.05605522 0.03657765 0.02563486]
Means and standard deviations for TPR, FDR and AUC with 0.25 time interval
[0.79375    0.49120444 0.5496875 ] [0.04532866 0.02250024 0.0106675 ]


In [4]:
# LiNGAM / SVAM performance with irregular data
for fraction in [1,0.7,0.5, 0.3]:
    perf = []
    for i in range(20):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC= simulate_lorenz_96(p, T=T, delta_t=0.1,sigma=0.05, F=10)

        # format for NeuralODE
        times = np.linspace(0, T, num_points)
        times_np = np.hstack([times[:, None]])
        #times = torch.from_numpy(times_np[:, :, None])
        data = torch.from_numpy(data[:, None, :].astype(np.float32))

        indeces = np.random.choice(range(T),int(fraction*T),replace=False)
        indeces = np.sort(indeces)

        data = compute_spline(data[indeces].squeeze().detach(), times[indeces], k=2,s=0)
        
        from source.lingam import lingam_method
        importlib.reload(utils)
        graph = lingam_method(data)
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('Means and standard deviations for TPR, FDR and AUC with', fraction, 'kept')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     

Means and standard deviations for TPR, FDR and AUC with 1 kept
[0.58875    0.13850842 0.69266667] [0.03208874 0.05396437 0.0164683 ]
Means and standard deviations for TPR, FDR and AUC with 0.7 kept
[0.595      0.15881673 0.68578125] [0.04227884 0.05779033 0.02153504]
Means and standard deviations for TPR, FDR and AUC with 0.5 kept
[0.5375     0.23101878 0.66558333] [0.06869316 0.07037457 0.02619013]
Means and standard deviations for TPR, FDR and AUC with 0.3 kept
[0.3275     0.513526   0.57895833] [0.04465143 0.04080761 0.02861502]


# DCM

In [5]:
# DCM performance with sparse data
for dt in [0.01,0.05,0.1,0.25,0.5,1]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC= simulate_lorenz_96(p, T=T, delta_t=dt,sigma=0.01, F=10)

        indeces = np.random.choice(range(T),int(T),replace=False)
        indeces = np.sort(indeces)

        from source.DCM import DCM_full
        graph = DCM_full(data[indeces],lambda1=0.05, s=4, w_threshold = 0.1)
        #plt.matshow(abs(graph),cmap='Reds')
        #plt.colorbar()
        #plt.show()
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('Means and standard deviations for TPR, FDR and AUC with', dt, 'time interval')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     

Means and standard deviations for TPR, FDR and AUC with 0.01 time interval
[0.003 0.    0.501] [0.008 0.    0.004]
Means and standard deviations for TPR, FDR and AUC with 0.05 time interval
[0.753 0.056 0.872] [0.008 0.028 0.005]
Means and standard deviations for TPR, FDR and AUC with 0.1 time interval
[0.922 0.157 0.949] [0.038 0.029 0.02 ]
Means and standard deviations for TPR, FDR and AUC with 0.25 time interval
[1.    0.599 0.942] [0.    0.002 0.027]
Means and standard deviations for TPR, FDR and AUC with 0.5 time interval
[1.    0.6   0.621] [0.000e+00 1.110e-16 4.653e-02]
Means and standard deviations for TPR, FDR and AUC with 1 time interval
[1.    0.6   0.526] [0.00e+00 1.11e-16 7.04e-02]


In [6]:
# DCM performance with irregular data
for fraction in [1,0.7,0.5, 0.3]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC= simulate_lorenz_96(p, T=T, delta_t=0.05,sigma=0.00, F=10)

        indeces = np.random.choice(range(T),int(fraction*T),replace=False)
        indeces = np.sort(indeces)

        #data = compute_spline(data[indeces].squeeze().detach(), times[indeces], k=2,s=0)

        from source.DCM import DCM_full
        graph = DCM_full(data[indeces],lambda1=0.05, s=4, w_threshold = 0.1)
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('Means and standatd deviations for TPR, FDR and AUC with', fraction, 'kept')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     

Means and standatd deviations for TPR, FDR and AUC with 1 kept
[0.753 0.055 0.872] [0.008 0.034 0.006]
Means and standatd deviations for TPR, FDR and AUC with 0.7 kept
[0.828 0.154 0.901] [0.031 0.035 0.014]
Means and standatd deviations for TPR, FDR and AUC with 0.5 kept
[0.945 0.336 0.937] [0.042 0.047 0.031]
Means and standatd deviations for TPR, FDR and AUC with 0.3 kept
[1.    0.596 0.823] [0.    0.006 0.046]


# PCMCI

In [7]:
# pcmci performance with sparse data
for dt in [0.01,0.05,0.1,0.15,0.25]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC= simulate_lorenz_96(p, T=T, delta_t=dt,sigma=0.05, F=10)

        # format for NeuralODE
        #times = np.linspace(0, T, num_points)
        #times_np = np.hstack([times[:, None]])
        #times = torch.from_numpy(times_np[:, :, None])
        #data = torch.from_numpy(data[:, None, :].astype(np.float32))

        indeces = np.random.choice(range(T),int(T),replace=False)
        indeces = np.sort(indeces)
        
        from source.pcmci import pcmci
        importlib.reload(utils)
        graph = pcmci(data[indeces])
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('TPR, FDR and AUC with', dt, 'time interval')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     

Could not import packages for CMIknn and GPDC estimation
Could not import rpy package
Could not import r-package RCIT
TPR, FDR and AUC with 0.01 time interval
[0.26  0.009 0.629] [0.017 0.027 0.009]
TPR, FDR and AUC with 0.05 time interval
[0.325 0.071 0.653] [0.039 0.064 0.017]
TPR, FDR and AUC with 0.1 time interval
[0.637 0.103 0.793] [0.064 0.066 0.032]
TPR, FDR and AUC with 0.15 time interval
[0.69  0.024 0.839] [0.03  0.027 0.017]
TPR, FDR and AUC with 0.25 time interval
[0.253 0.496 0.545] [0.063 0.08  0.034]


In [8]:
# pcmci performance with irregular data
for fraction in [1,0.7,0.5, 0.3]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC= simulate_lorenz_96(p, T=T, delta_t=0.1,sigma=0.05, F=10)

        # format for NeuralODE
        times = np.linspace(0, T, num_points)
        times_np = np.hstack([times[:, None]])
        #times = torch.from_numpy(times_np[:, :, None])
        data = torch.from_numpy(data[:, None, :].astype(np.float32))

        indeces = np.random.choice(range(T),int(fraction*T),replace=False)
        indeces = np.sort(indeces)

        data = compute_spline(data[indeces].squeeze().detach(), times[indeces], k=2,s=0)

        from source.pcmci import pcmci
        importlib.reload(utils)
        graph = pcmci(data)
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('TPR, FDR and AUC with', dt, 'time interval')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     

TPR, FDR and AUC with 0.25 time interval
[0.672 0.114 0.808] [0.088 0.027 0.046]
TPR, FDR and AUC with 0.25 time interval
[0.408 0.12  0.684] [0.058 0.063 0.023]
TPR, FDR and AUC with 0.25 time interval
[0.325 0.129 0.646] [0.043 0.072 0.024]
TPR, FDR and AUC with 0.25 time interval
[0.35  0.154 0.654] [0.055 0.079 0.036]


## NGM

In [ ]:
# NGM performance with sparse data
import warnings
warnings.filterwarnings("ignore")
for dt in [0.01,0.05,0.1,0.15,0.25]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC= simulate_lorenz_96(p, T=T, delta_t=dt,sigma=0.05, F=10)

        # format for NeuralODE
        times = np.linspace(0, T, num_points)
        times_np = np.hstack([times[:, None]])
        times = torch.from_numpy(times_np[:, :, None].astype(np.float32))
        data = torch.from_numpy(data[:, None, :].astype(np.float32))

        import source.NMC as models
        func = models.MLPODEF(dims=[p, 12, 1],GL_reg=0.1)
        
        # GL training
        models.train(func,data,n_steps=1000,plot = False, plot_freq=20)
        # AGL training
        weights = func.group_weights()
        func.GL_reg *= (1 / weights)
        func.reset_parameters()
        models.train(func,data,n_steps=1000,plot = False, plot_freq=20)
        graph = func.causal_graph(w_threshold=0.1)
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('Means and standard deviations for TPR, FDR and AUC with', dt, 'time interval')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     

In [ ]:
# NGM performance with irregular data
for fraction in [1, 0.7, 0.5, 0.3]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 3000
        num_points = T
        data, GC= simulate_lorenz_96(p, T=T, delta_t=0.05,sigma=0.05, F=5)

        # format for NeuralODE
        indeces = np.random.choice(range(T),int(fraction*T),replace=False)
        indeces = np.sort(indeces)
        times = np.linspace(0, T, num_points)
        times_np = np.hstack([times[indeces, None]])
        data = data[indeces,:]
        times = torch.from_numpy(times_np[:, :, None])
        data = torch.from_numpy(data[:, None, :].astype(np.float32))
        
        import source.NMC as models
        func = models.MLPODEF(dims=[p, 12, 1],GL_reg=0.05)
        models.train(func,data,n_steps=2000, horizon = 5, plot_freq=20, plot=True, irregular=True, times=times)
        #weights = func.group_weights()
        #func.GL_reg *= (1 / weights)
        #func.reset_parameters()
        #models.train(func,data,n_steps=1000,horizon = 5, plot_freq=20, plot=False, irregular=True, times=times)
        graph = func.causal_graph(w_threshold=0.09)
        print(graph)
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('Means and standard deviations for TPR, FDR and AUC with', fraction, 'kept')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     